In [ ]:
%qtconsole

# Setup and Calibration

## import library

In [ ]:
from __future__ import print_function

import time

import odrive
import odrive.utils
import odrive.enums
from odrive.enums import *

import math
import matplotlib.pyplot as plt

%matplotlib inline

## forward kinematics

In [ ]:
import numpy as np

import matplotlib as mpl

import matplotlib.pyplot as plt
plt.ion()

# for the symbolic manipulation of jacobian
import sympy as sym
from sympy import init_printing
init_printing() # doctest: +SKIP
from sympy.utilities.lambdify import lambdify

from scipy.optimize import minimize
from scipy.optimize import fsolve

theta_L, theta_R = sym.symbols('theta_L, theta_R')
x, y = sym.symbols('x y')

link_1 = 0.09
link_2 = 0.16
w = 0.07

gx = w+link_1*sym.cos(theta_R)-link_1*sym.cos(theta_L)
gy = link_1*sym.sin(theta_R)-link_1*sym.sin(theta_L)
g = sym.sqrt(gx**2+gy**2)

cosA = g/(2*link_2)
sinA = sym.sqrt(1-cosA**2)

cosg = gx/g
sing = gy/g

# x = link_1*sym.cos(theta_1)+link_2*sym.cos(theta_A+theta_g)
x = link_1*sym.cos(theta_L)+link_2*(cosA*cosg-sinA*sing)
x_toe = x -w/2 # move the origin to (w/2,0)
x_toe = x_toe.simplify()

# y = link_1*sym.sin(theta_1)+link_2*sym.sin(theta_A+theta_g)
y = link_1*sym.sin(theta_L)+link_2*(sinA*cosg+cosA*sing)
y_toe = y.simplify()

# Forward Kinematics
FK_position = sym.Matrix([[x_toe], [y_toe]])
FK_position

# Jacobian
Jacobian = FK_position.jacobian([theta_R, theta_L])
# Jacobian.simplify() # this simplify step takes pretty long...
Jacobian

Jacobian_fast = lambdify((theta_R, theta_L), Jacobian)
FK_position_fast = lambdify((theta_R, theta_L), FK_position)

# test FK with Isosceles triangle case
sym.sqrt(25**2-3.5**2)
np.pi-sym.asin(24.7537/25)

print('FK test, theta_R = 1.71, theta_L = 1.43')
print(FK_position_fast(1.71, 1.43))
print('FK test, theta_R = 0, theta_L = pi')
print(FK_position_fast(0, np.pi))
# print('FK test, theta_R = 0.5, theta_L = pi')
# print(FK_position_fast(0.5, np.pi))

# from IPython.display import Image
# Image("img/Screen Shot 2019-06-09 at 11.22.53 PM.png")


## connect to odrive

In [ ]:
odrvR = odrive.find_any(serial_number = "3065394D3235")
if odrvR is not None:
    print('Right Side Motors Connected!')
    print('Odrive serial {}'.format(odrvR.serial_number))
else:
    print('Not connected')
    
time.sleep(1)

odrvL = odrive.find_any(serial_number = "306139573235")
if odrvL is not None:
    print('Left Side Motors Connected!')
    print('Odrive serial {}'.format(odrvL.serial_number))
else:
    print('Not connected')

## calibration

In [ ]:
def Calibration (odrv0):
    odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE # right motor
    odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE # left motor
    print('calibration done')
    return ()


In [ ]:
Calibration(odrvR)
time.sleep(1)
Calibration(odrvL)


## set parameters

In [ ]:
def SetParameter(odrv0):
    odrv0.axis0.controller.config.vel_limit = 200000
    odrv0.axis1.controller.config.vel_limit = 200000
    odrv0.axis0.controller.config.vel_limit_tolerance = 10000
    odrv0.axis1.controller.config.vel_limit_tolerance = 10000
    odrv0.axis0.motor.config.requested_current_range = 90
    odrv0.axis1.motor.config.requested_current_range = 90
    odrv0.axis0.controller.config.pos_gain = 20
    odrv0.axis1.controller.config.pos_gain = 20
    print('parameter set')
    return ()

def GetEncoderState(odrv0):
    position_cpr0 = odrv0.axis0.encoder.pos_estimate # right motor
    position_cpr1 = odrv0.axis1.encoder.pos_estimate # left motor
    return (position_cpr0,position_cpr1)

def IdleMode(odrv0):
    # necessary to switch back to idle to manually control the links to read the encoder position
    odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
    odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE
    return()

def ClosedLoopControlMode(odrv0):
    # enter closed-loop control to give command to the motor
    odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
    return()

def MotorClearError(odrv0):
    odrv0.axis0.error = 0
    odrv0.axis0.motor.error = 0
    odrv0.axis0.controller.error = 0
    odrv0.axis0.encoder.error = 0
    odrv0.axis1.error = 0
    odrv0.axis1.motor.error = 0
    odrv0.axis1.controller.error = 0
    odrv0.axis1.encoder.error = 0
    return()

def CheckMotorError(odrv0):
    axis0error = odrv0.axis0.error 
    axis0motorerror = odrv0.axis0.motor.error 
    axis0controllererror = odrv0.axis0.controller.error 
    axis1error = odrv0.axis1.error 
    axis1motorerror = odrv0.axis1.motor.error 
    axis1controllererror = odrv0.axis1.controller.error 
    return(axis0error, axis0motorerror, axis0controllererror, axis1error, axis1motorerror, axis1controllererror)


In [ ]:
SetParameter(odrvR)
SetParameter(odrvL)


## define home positions

In [ ]:
RM_home_position = GetEncoderState(odrvR)
RM_homeR = RM_home_position[0] # right motor, M0
RM_homeL = RM_home_position[1] # left motor, M1
print('right side motors home_position')
print('right, left')
print(RM_home_position)

LM_home_position = GetEncoderState(odrvL)
LM_homeR = LM_home_position[0] # right motor, M0
LM_homeL = LM_home_position[1] # left motor, M1
print('left side motors home_position')
print('right, left')
print(LM_home_position)

# Demo 1: Input desired trajectory by moving the end effector

## read the encoder position to generate trajectory

In [ ]:
def ReadManuallyInput(odrv0, homeR, homeL, timelength):
    left_motor_en = [] # motor M1, position in encoder unit
    left_motor_rad = [] # motor M1, position in rad 
    right_motor_en = [] # motor M0, position in encoder unit
    right_motor_rad = [] # motor M0, position in rad 
    import time
    timeout = time.time() + timelength 
    while True:
        test = 0
        if test == 15 or time.time() > timeout:
            break
        test = test - 1
        # first read the encoder value
        encoder_state = GetEncoderState(odrv0)
#         print(encoder_state)
        right_motor_en.append(encoder_state[0])
        left_motor_en.append(encoder_state[1])
        # then transform into radius
        theta_R = -(encoder_state[0]-homeR)*2*np.pi/8192
        theta_L = np.pi-(encoder_state[1]-homeL)*2*np.pi/8192 
#         print(theta_R, theta_L)
        if theta_R > np.pi:
            theta_R = theta_R-2*np.pi
        elif theta_R < -np.pi:
            theta_R = theta_R + 2*np.pi
        else:
            theta_R = theta_R
        if theta_L > np.pi:
            theta_L = theta_L-2*np.pi
        elif theta_L < -np.pi:
            theta_R = theta_L + 2*np.pi
        else:
            theta_L = theta_L
        right_motor_rad.append(theta_R)
        left_motor_rad.append(theta_L)
        time.sleep(0.01)
    print('loop end, trajectory recorded')
    return(right_motor_rad, left_motor_rad, right_motor_en, left_motor_en)

## test left side motors 

## read trajectory

In [ ]:
timelength = 5
LMangle = ReadManuallyInput(odrvL, LM_homeR, LM_homeL, timelength)
right_motor_rad = []
left_motor_rad = []
right_motor_rad = LMangle[0]
left_motor_rad = LMangle[1]

x_plot = []
y_plot = []
for i in range(len(right_motor_rad)):
    xy_position=FK_position_fast(right_motor_rad[i], left_motor_rad[i])
    x_plot.append(xy_position[0])
    y_plot.append(xy_position[1])
    
plt.plot(x_plot, y_plot)
plt.xlabel('x position (m)')
plt.ylabel('y position (m)')


## open loop control

In [ ]:
# enter closed-loop control to give command to the motor
ClosedLoopControlMode(odrvL)

right_motor_en = []
left_motor_en = []

# Generate a cyclic leg trajectory
right_motor_en = LMangle[2]
left_motor_en = LMangle[3]
for i in range(len(left_motor_en)):
    odrvL.axis0.controller.pos_setpoint = right_motor_en[i]
    odrvL.axis1.controller.pos_setpoint = left_motor_en[i]
    time.sleep(0.01)

# end the loop with the motor back to idle mode
MotorClearError(odrvL)
IdleMode(odrvL)

## test right side motors

## read trajectory

In [ ]:
timelength = 5
RMangle = ReadManuallyInput(odrvR, RM_homeR, RM_homeL, timelength)
right_motor_rad = RMangle[0]
left_motor_rad = RMangle[1]

x_plot = []
y_plot = []
for i in range(len(right_motor_rad)):
    xy_position=FK_position_fast(right_motor_rad[i], left_motor_rad[i])
#     print(xy_position)
    x_plot.append(xy_position[0])
    y_plot.append(xy_position[1])
    
plt.plot(x_plot, y_plot)
plt.xlabel('x position (m)')
plt.ylabel('y position (m)')


## open loop control

In [ ]:
# enter closed-loop control to give command to the motor
ClosedLoopControlMode(odrvR)

# Generate a cyclic leg trajectory
right_motor_en = RMangle[2]
left_motor_en = RMangle[3]
for i in range(len(left_motor_en)):
    odrvR.axis0.controller.pos_setpoint = right_motor_en[i]
    odrvR.axis1.controller.pos_setpoint = left_motor_en[i]
    time.sleep(0.01)

# end the loop with the motor back to idle mode
# MotorClearError(odrvR)
IdleMode(odrvR)

# Demo 2: Using IK to generate desired trajectory

## inverse kinematics

In [ ]:
l1 = 0.09;                  # m 
l2 = 0.16;                  # m
w = 0.07;                   # m

def IK_5_link(x, y, l1 = l1, l2 = l2, w = w):
    
    def leg_wide(var):
        return np.linalg.norm([var[0], var[1] - np.pi])
    
    def x_constraint_equation(var):
        # should be equal to zero when the 
        return l1**2 - l2**2 + (x - w/2)**2 + y**2 - 2*l1*(y*np.sin(var[0]) + (x - w/2)*np.cos(var[0]))

    def y_constraint_equation(var):
        return l1**2 - l2**2 + (x + w/2)**2 + y**2 - 2*l1*(y*np.sin(var[1]) + (x + w/2)*np.cos(var[1]))

    
    res = minimize(leg_wide, (0.1, 9*np.pi/10), method="SLSQP", \
                   constraints= ({"type": "eq", "fun": x_constraint_equation}, \
                                 {"type": "eq", "fun": y_constraint_equation}))
    
    return (res, np.linalg.norm([x_constraint_equation(res.x), y_constraint_equation(res.x)]))


# Test, the following theta's correspond to the x-y below
thetaR = .5
thetaL = np.pi
    
# x = -0.024021708847354217
# y = 0.12411037295149752
# x = 0
# y = 9.98749218e-02
x=  0
y = 2.47537816e-01

res = IK_5_link(x, y)


print("""Compare the FK position (top) and the IK solution (bottom) method: 
          \r\n theta_R = {:.4f} \t theta_L = {:.4f} \r\n theta_R = {:.4f} \t theta_L = {:.4f}""".format(thetaR, thetaL, res[0].x[0], res[0].x[1]))

## parabola swing phase trajectory

In [ ]:
stance_phase_touch = 0.10
stance_phase_lift = -0.10
stance_phase_height = 0.18
swing_phase_height = 0.10

swing_step = 0.002
stance_step = 0.004

# from IPython.display import Image
# Image("img/Screen Shot 2019-06-09 at 11.34.31 PM.png")

In [ ]:
def calc_parabola_vertex(x1, y1, x2, y2, x3, y3):

    denom = (x1-x2) * (x1-x3) * (x2-x3);
    A     = (x3 * (y2-y1) + x2 * (y1-y3) + x1 * (y3-y2)) / denom;
    B     = (x3*x3 * (y1-y2) + x2*x2 * (y3-y1) + x1*x1 * (y2-y3)) / denom;
    C     = (x2 * x3 * (x2-x3) * y1+x3 * x1 * (x3-x1) * y2+x1 * x2 * (x1-x2) * y3) / denom;

    return (A,B,C)

x1,y1=[stance_phase_touch,stance_phase_height]
x2,y2=[stance_phase_lift,stance_phase_height]
x3,y3=[0,swing_phase_height]

#Calculate the unknowns of the equation y=ax^2+bx+c
a,b,c=calc_parabola_vertex(x1, y1, x2, y2, x3, y3)


# swing phase
x_swing_pos_temp=np.arange(stance_phase_lift,stance_phase_touch,swing_step)
y_swing_pos_temp=[]
#Calculate y values 
for x in range(len(x_swing_pos_temp)):
    x_val=x_swing_pos_temp[x]
    y=(a*(x_val**2))+(b*x_val)+c
    y_swing_pos_temp.append(y)
    
RMx_swing_pos_temp=-np.arange(stance_phase_lift,stance_phase_touch,swing_step)
RMy_swing_pos_temp=[]
#Calculate y values 
for x in range(len(RMx_swing_pos_temp)):
    x_val=RMx_swing_pos_temp[x]
#     RMx_swing_pos_temp.append(x_val)
    y=(a*(x_val**2))+(b*x_val)+c
    RMy_swing_pos_temp.append(y)

    
# stance phase
x_stance_pos_temp=-np.arange(stance_phase_lift,stance_phase_touch,stance_step)
y_stance_pos_temp=[]
#Calculate y values 
for x in range(len(x_stance_pos_temp)):
#     x_val=x_swing_pos_temp[x]
    y=stance_phase_height
    y_stance_pos_temp.append(y)
    
RMx_stance_pos_temp=np.arange(stance_phase_lift,stance_phase_touch,stance_step)
RMy_stance_pos_temp=[]
#Calculate y values 
for x in range(len(RMx_stance_pos_temp)):
    x_val=RMx_swing_pos_temp[x]
#     RMx_stance_pos_temp.append(x_val)
    y=stance_phase_height
    RMy_stance_pos_temp.append(y)


plt.plot(x_swing_pos_temp, y_swing_pos_temp, linestyle='-.', color='black') # parabola line
plt.scatter(x_swing_pos_temp, y_swing_pos_temp, color='gray') # parabola points
plt.plot(x_stance_pos_temp, y_stance_pos_temp, linestyle='-.', color='red') # parabola line
plt.scatter(x_stance_pos_temp, y_stance_pos_temp, color='gray') # parabola points
plt.show()


def IKtoEncoderPosition(homeR, homeL, x_swing_pos_temp, y_swing_pos_temp, x_stance_pos_temp, y_stance_pos_temp):

    theta_R_IK_temp = []
    theta_L_IK_temp = []
    encoder_right_IK_temp = []
    encoder_left_IK_temp = []

    # swing phase
    for i in range(len(x_swing_pos_temp)):
        x = x_swing_pos_temp[i]
        y = y_swing_pos_temp[i]
        res = IK_5_link(x, y)
        theta_R_IK = np.array(res[0].x[0])
        theta_L_IK = np.array(res[0].x[1])
        theta_R_IK_temp.append(theta_R_IK)
        theta_L_IK_temp.append(theta_L_IK)
        encoder_right_IK = homeR + 8192/(2*np.pi)*(-theta_R_IK)
        encoder_left_IK = homeL + 8192/(2*np.pi)*(np.pi-theta_L_IK)
        encoder_right_IK_temp.append(encoder_right_IK)
        encoder_left_IK_temp.append(encoder_left_IK)

    # stance phase
    for j in range(len(x_stance_pos_temp)):
        x = x_stance_pos_temp[j]
        y = y_stance_pos_temp[j]
        res = IK_5_link(x, y)
        theta_R_IK = np.array(res[0].x[0])
        theta_L_IK = np.array(res[0].x[1])
        theta_R_IK_temp.append(theta_R_IK)
        theta_L_IK_temp.append(theta_L_IK)
        encoder_right_IK = homeR + 8192/(2*np.pi)*(-theta_R_IK)
        encoder_left_IK = homeL + 8192/(2*np.pi)*(np.pi-theta_L_IK)
        encoder_right_IK_temp.append(encoder_right_IK)
        encoder_left_IK_temp.append(encoder_left_IK)
        
    return(encoder_right_IK_temp, encoder_left_IK_temp )

## test right side motors

In [ ]:
# RMangle = ReadManuallyInput(odrvR, RM_homeR, RM_homeL, timelength)
RM_IKdata = IKtoEncoderPosition(RM_homeR, RM_homeL, RMx_swing_pos_temp, RMy_swing_pos_temp, RMx_stance_pos_temp, RMy_stance_pos_temp)

RMright_motor_en = []
RMleft_motor_en = []
RMright_motor_en = RM_IKdata[0]
RMleft_motor_en = RM_IKdata[1]

# ClosedLoopControlMode(odrvR)
# looptime = 5
# t0 = time.time()
# i = 0
# while True:
#     odrvR.axis0.controller.pos_setpoint = RMright_motor_en[i]
#     odrvR.axis1.controller.pos_setpoint = RMleft_motor_en[i]
#     time.sleep(0.01)
#     i = i+1
#     if i>len(LMleft_motor_en)-1:
#         i = 0
#     if time.time()-t0 > looptime:
#         break

# # end the loop with the motor back to idle mode
# MotorClearError(odrvR)
# IdleMode(odrvR)


In [ ]:
len(RMright_motor_en)
len(RMleft_motor_en)
# RMleft_motor_en

In [ ]:
ClosedLoopControlMode(odrvR)
looptime = 5
t0 = time.time()
i = 0
while True:
    odrvR.axis0.controller.pos_setpoint = RMright_motor_en[i]
    odrvR.axis1.controller.pos_setpoint = RMleft_motor_en[i]
    time.sleep(0.01)
    i = i+1
    if i>len(LMleft_motor_en)-1:
        i = 0
    if time.time()-t0 > looptime:
        break

# end the loop with the motor back to idle mode
MotorClearError(odrvR)
IdleMode(odrvR)


## test left side motors

In [ ]:
# RMangle = ReadManuallyInput(odrvR, RM_homeR, RM_homeL, timelength)
LM_IKdata = IKtoEncoderPosition(LM_homeR, LM_homeL, x_swing_pos_temp, y_swing_pos_temp, x_stance_pos_temp, y_stance_pos_temp)

# Generate a cyclic leg trajectory
LMright_motor_en = []
LMleft_motor_en = []
LMright_motor_en = LM_IKdata[0]
LMleft_motor_en = LM_IKdata[1]

# # ClosedLoopControlMode(odrvL)
# ClosedLoopControlMode(odrvL)
# looptime = 5
# t0 = time.time()
# i = 0
# while True:
# #     for i in range(len(LMleft_motor_en)):
#     odrvL.axis0.controller.pos_setpoint = LMright_motor_en[i]
#     odrvL.axis1.controller.pos_setpoint = LMleft_motor_en[i]
#     time.sleep(0.01)
#     i = i+1
#     if i>len(LMleft_motor_en)-1:
#         i = 0
#     if time.time()-t0 > looptime:
#         break


# # end the loop with the motor back to idle mode
# MotorClearError(odrvL)
# IdleMode(odrvL)

In [ ]:
len(LMleft_motor_en)

In [ ]:
# ClosedLoopControlMode(odrvL)
ClosedLoopControlMode(odrvL)
looptime = 20
t0 = time.time()
i = 0
while True:
#     for i in range(len(LMleft_motor_en)):
    odrvL.axis0.controller.pos_setpoint = LMright_motor_en[i]
    odrvL.axis1.controller.pos_setpoint = LMleft_motor_en[i]
    time.sleep(0.01)
    i = i+1
    if i>len(LMleft_motor_en)-1:
        i = 0
    if time.time()-t0 > looptime:
        break


# end the loop with the motor back to idle mode
MotorClearError(odrvL)
IdleMode(odrvL)

# final test with both legs

In [ ]:
RM_homeL = 0
RM_homeR = 0
LM_homeL = 0
LM_homeR = 0

In [ ]:
# RMangle = ReadManuallyInput(odrvR, RM_homeR, RM_homeL, timelength)
RM_IKdata = IKtoEncoderPosition(RM_homeR, RM_homeL, RMx_swing_pos_temp, RMy_swing_pos_temp, RMx_stance_pos_temp, RMy_stance_pos_temp)

RMright_motor_en = []
RMleft_motor_en = []
RMright_motor_en = RM_IKdata[0]
RMleft_motor_en = RM_IKdata[1]

# RMangle = ReadManuallyInput(odrvR, RM_homeR, RM_homeL, timelength)
LM_IKdata = IKtoEncoderPosition(LM_homeR, LM_homeL, x_swing_pos_temp, y_swing_pos_temp, x_stance_pos_temp, y_stance_pos_temp)

# Generate a cyclic leg trajectory
LMright_motor_en = []
LMleft_motor_en = []
LMright_motor_en = LM_IKdata[0]
LMleft_motor_en = LM_IKdata[1]

In [ ]:
a =len(RMright_motor_en)
b= len(LMleft_motor_en)
print('number of data in one cycle')
print(a,b)

# make sure the leading phase < len(encoder array)

In [ ]:
ClosedLoopControlMode(odrvR)
ClosedLoopControlMode(odrvL)
looptime = 5
t0 = time.time()
i = 0
leading_phase = 30
print("time period(sec), phase difference (degree)")
phase_difference = leading_phase/len(RMright_motor_en)*360
time_period = 0.01*len(RMright_motor_en)
print(time_period, phase_difference)

while True:
    odrvR.axis0.controller.pos_setpoint = RMright_motor_en[leading_phase]
    odrvR.axis1.controller.pos_setpoint = RMleft_motor_en[leading_phase]
    odrvL.axis0.controller.pos_setpoint = LMright_motor_en[i]
    odrvL.axis1.controller.pos_setpoint = LMleft_motor_en[i]
    time.sleep(0.01)
    i = i+1
    leading_phase = leading_phase+1
    if i>len(LMleft_motor_en)-1:
        i = 0
    if leading_phase>len(RMright_motor_en)-1:
        leading_phase = 0
    if time.time()-t0 > looptime:
        break

# end the loop with the motor back to idle mode
MotorClearError(odrvR)
IdleMode(odrvR)
MotorClearError(odrvL)
IdleMode(odrvL)